# **Security data science**

### Laboratorio 2

In [ ]:
import pandas as pd
import numpy as np
import  re
import  matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import contractions
import nltk

from urllib.parse import urlparse, urlencode
from pandas_profiling import ProfileReport
from sklearn import metrics, model_selection, tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

pd.options.display.max_colwidth = 200

In [ ]:
!pip install contractions #xpansion de contracciones
!pip install textsearch

## **Parte 1**

### Exploracion de datos

In [ ]:
df1 = pd.read_csv('completeSpamAssassin.csv') #primer datas set
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6046 entries, 0 to 6045
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6046 non-null   int64 
 1   Body        6045 non-null   object
 2   Label       6046 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 141.8+ KB


In [8]:
df2 = pd.read_csv('enronSpamSubset.csv') #segundo data set
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    10000 non-null  int64 
 1   Unnamed: 0.1  10000 non-null  int64 
 2   Body          10000 non-null  object
 3   Label         10000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 312.6+ KB


In [ ]:
df1 = df1.drop(['Unnamed: 0'], axis=1)
df1.head()

In [ ]:
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df2.head()

#### Unificacion de data set

In [11]:
df = pd.concat([df1, df2], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16046 entries, 0 to 16045
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Body    16045 non-null  object
 1   Label   16046 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.8+ KB


### Preprocesamiento

In [ ]:
df['Body'] = df['Body'].str.lower() #conversion de mayusculas y minusculas
df.head()

In [ ]:
df['Body'] = df['Body'].str.strip()
df.head()

In [16]:
def removerAcentos(texto):
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ascii','ignore').decode('utf-8','ignore') #eliminacion de acentos
    return texto

In [ ]:
df['Body'] = df['Body'].apply(removerAcentos)
df.head()

In [ ]:
expan = []
for col in df["Body"]:
    expan.append(contractions.fix(col))

df["Body"] = expan
df.head()

In [ ]:
print(nltk.word_tokenize(df["Body"][0])) #eliminacion de stop words

In [23]:
tokens = []
for row in df["Body"]:
    token = nltk.word_tokenize(row)
    tokens.append(token)

In [ ]:
stop_words_spanish = nltk.corpus.stopwords.words('spanish')
tokens_filtrados_es = [token for token in tokens if token not in stop_words_spanish]
tokens_filtrados_es

In [ ]:
stop_words_english = nltk.corpus.stopwords.words('english')
tokens_filtrados_en = [token for token in tokens if token not in stop_words_english]
tokens_filtrados_en

In [26]:
def removerCaracteresEspecialesNumerosSimbolos(texto, removerDigitos = False): #eliminacion de caracteres especiales
    patron = r'[^a-zA-Z0-9\s]' if not removerDigitos else r'[^a-zA-Z\s]'
    texto = re.sub(patron, '', texto, re.I|re.A)
    return texto

In [ ]:
df['Body'] = df['Body'].apply(removerCaracteresEspecialesNumerosSimbolos)
df.head()

### Representacion de texto


In [28]:
def normalizeDoc(doc):
    tokens = nltk.word_tokenize(doc) #documento token
    filtered_tokens = [token for token in tokens if token not in stop_words_english] #filtracion palabras vacias
    doc = ' '.join(filtered_tokens) #re creacion de documento con tokens
    return doc

In [ ]:
df['Body'] = df['Body'].apply(normalizeDoc)
df.head(10)

### BoG (para n = 1,2)

In [32]:
cv = CountVectorizer(min_df=0.1, max_df=0.9)
cv_matrix = cv.fit_transform(df['Body'])
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 2, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [33]:
vocabulario = cv.get_feature_names() #palabras unicas

cv_df = pd.DataFrame(cv_matrix, columns=vocabulario) #vector
cv_df

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
0,0,0,0,0,3,0,0,2,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,3,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16042,5,0,0,0,0,0,5,0,0,0,...,2,5,0,3,0,0,0,0,0,0
16043,1,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
16044,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [35]:
bv = CountVectorizer(ngram_range=(1,2), min_df=0.1, max_df=0.9)
bv_matrix = bv.fit_transform(df['Body'])

bv_matrix = bv_matrix.toarray()
vocabulario = bv.get_feature_names()
bv_df = pd.DataFrame(bv_matrix, columns=vocabulario)
bv_df

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
0,0,0,0,0,3,0,0,2,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,3,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16042,5,0,0,0,0,0,5,0,0,0,...,2,5,0,3,0,0,0,0,0,0
16043,1,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
16044,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### TF-IDF

In [36]:
tv = TfidfVectorizer(min_df=0.1, max_df=0.9, use_idf=True)
tv_matrix = tv.fit_transform(df['Body'])
tv_matrix = tv_matrix.toarray()

vocabulario = tv.get_feature_names()
tv_df = pd.DataFrame(np.round(tv_matrix, 2), columns=vocabulario)
tv_df

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
0,0.00,0.00,0.0,0.00,0.5,0.0,0.00,0.34,0.00,0.00,...,0.00,0.0,0.0,0.15,0.00,0.00,0.0,0.0,0.00,0.00
1,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.32,0.00,0.0,0.0,0.00,0.00
2,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.43,0.00,0.0,0.0,0.00,0.00
3,0.00,0.00,0.0,0.00,0.2,0.0,0.00,0.07,0.00,0.00,...,0.00,0.0,0.0,0.06,0.00,0.00,0.0,0.0,0.00,0.00
4,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.33,0.00,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16041,0.56,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
16042,0.35,0.00,0.0,0.00,0.0,0.0,0.38,0.00,0.00,0.00,...,0.14,0.3,0.0,0.19,0.00,0.00,0.0,0.0,0.00,0.00
16043,0.21,0.48,0.0,0.00,0.0,0.0,0.23,0.00,0.00,0.00,...,0.00,0.0,0.0,0.00,0.00,0.22,0.0,0.0,0.23,0.00
16044,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0.54,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00


## **Parte 2**

### Separacion de datos


### Implementacion
